In [8]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

In [9]:
data = pd.read_csv('./dataset/data.csv')
data.drop("Unnamed: 0", axis=1, inplace=True)

In [10]:
data.tail(10)

,end_date,sold_units
211,2024-01-20,143.0
212,2024-01-27,238.0
213,2024-02-03,190.0
214,2024-02-10,145.0
215,2024-02-17,126.0
216,2024-02-24,198.0
217,2024-03-02,149.0
218,2024-03-09,148.0
219,2024-03-16,146.0
220,2024-03-23,100.0


In [11]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class Dataset_ean(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='data.csv',
                 target='sold_units', scale=True, timeenc=0, freq='W'):
        if size is None:
            self.seq_len = 13  # Use 0.25 year of data for sequence
            self.label_len = 4  # Labels from last month
            self.pred_len = 4   # Predict 1 month ahead
        else:
            self.seq_len, self.label_len, self.pred_len = size

        assert flag in ['train', 'test', 'val']
        self.set_type = {'train': 0, 'val': 1, 'test': 2}[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()
        self.enc_in = self.data_x.shape[-1]
    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path, self.data_path))
        df_raw.drop("Unnamed: 0", axis=1, inplace=True)# don't forget to remove this once regulated the data
        num_weeks = len(df_raw)
        num_train = int(num_weeks * 0.7)
        num_val = int(num_weeks * 0.2)
        num_test = num_weeks - num_train - num_val

        border1s = [0, num_train, num_train + num_val]
        border2s = [num_train, num_train + num_val, num_weeks]

        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.scale:
            train_data = df_raw.iloc[:num_train]
            self.scaler.fit(train_data[[self.target]].values)
            data = self.scaler.transform(df_raw[[self.target]].values)
        else:
            data = df_raw[[self.target]].values

        df_stamp = pd.to_datetime(df_raw.iloc[:, 0][border1:border2])  # 'end_date' is first column
        if self.timeenc == 0:
            time_features = np.vstack((df_stamp.dt.year, df_stamp.dt.month, df_stamp.dt.day, df_stamp.dt.weekday)).T
        elif self.timeenc == 1:
            time_features = self.custom_time_features(df_stamp, freq=self.freq)

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.time_features = time_features

    def custom_time_features(self, dates, freq):
        # Define custom time feature extraction based on your model's needs
        return np.array([dates.index.weekday, dates.index.day])

    def __getitem__(self, index):
        seq_x = self.data_x[index:index+self.seq_len]
        seq_y = self.data_y[index+self.seq_len:index+self.seq_len+self.pred_len]
        seq_x_mark = self.time_features[index:index+self.seq_len]
        seq_y_mark = self.time_features[index+self.seq_len:index+self.seq_len+self.pred_len]
        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return (len(self.data_x) - self.seq_len - self.pred_len + 1) * self.enc_in

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)


In [12]:
root_path = './dataset'

In [13]:
data = Dataset_ean(root_path=root_path, scale=False, flag = 'test')

In [14]:
len(data.data_x), data.seq_len, data.pred_len

(23, 13, 4)

In [15]:
len(data)

7

In [11]:
seq_x, seq_y, seq_x_mark, seq_y_mark = data[0]

In [92]:
from data_provider.data_factory import data_provider

In [13]:
from torch.utils.data import DataLoader

In [21]:
data_loader = DataLoader(data)

In [22]:
print(len(data_loader))

7


In [2]:
import torch
import os

In [4]:
path = 'checkpoints/long_term_forecast_EAN_512_96_TimeLLM_ean_ftS_sl13_ll1_pl1_dm32_nh8_el2_dl1_df128_fc3_ebtimeF_Exp_0-TimeLLM-EAN/checkpoint'

# Load the checkpoint
checkpoint = torch.load(path)

In [5]:
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

NameError: name 'model' is not defined